In [1]:
import numpy as np
import os
from sklearn.model_selection import train_test_split

import tensorflow as tf
import csv

import pandas as pd
import pixiedust
import pdb

from sklearn.metrics import accuracy_score

def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed
                  )
from sklearn.preprocessing import StandardScaler

#PREPARING FOR TENSORBOARD:
from datetime import datetime
#FOR DIFFERENT MODELS MAKE DIFFERENT LOG NAME!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!1
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/DNN_simple-{}/".format(root_logdir, now)

Pixiedust database opened successfully


In [2]:
# %%pixie_debugger
print ('DATA PREPARATION')
vege_csv = csv.reader(open('merged1301_clean.csv', newline=''), delimiter=' ', quotechar='|')

vege_array = []

for row in vege_csv:
	red_array = []
	items = row[0].split(';')
	for i in items:
		red_array.append(i)

	vege_array.append(red_array)

klase = []
values = []

vege_array_fix = vege_array[:-4]

print ('length of the fixed array is ', len (vege_array_fix))

for r in vege_array_fix:
    value_row = []
    i = 0
    while i < len(r):
        if i == 0:
#             klase.append(float(r[i]))
            
            if r[i] == '0':
                klase.append(0)
            if r[i] == '1':
                klase.append(1)
            if r[i] == '2':
                klase.append(2)
        else:
            value_row.append(float(r[i]))
        i = i + 1
    values.append(value_row)

print ('test : ', values[10000])
print ('number of value arrays is: ', len(values), ' and number of classes is: ', len(klase))

DATA PREPARATION
length of the fixed array is  3651826
test :  [305.0, 162.0, 188.0, 184.0, 114.0, 438.0, 738.0, 698.0]
number of value arrays is:  3651826  and number of classes is:  3651826


In [3]:
X_train, X_test, y_train, y_test = train_test_split(values, klase, test_size=0.20, random_state=42)

scaler = StandardScaler()
X_train = np.array(X_train) #.astype('float32')
X_train = scaler.fit_transform(X_train.astype(np.float64))

In [4]:
y_train

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 2,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 2,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 1,
 0,
 1,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [ ]:
he_init = tf.variance_scaling_initializer()

def dnn(inputs, n_hidden_layers=5, n_neurons=100, name=None,
        activation=tf.nn.elu, initializer=he_init):
    with tf.variable_scope(name, "dnn"):
        for layer in range(n_hidden_layers):
            inputs = tf.layers.dense(inputs, n_neurons, activation=activation,
                                     kernel_initializer=initializer,
                                     name="hidden%d" % (layer + 1))
        return inputs

In [ ]:
n_channels = 8
n_outputs = 3
height = 1
width = 1

#not sure if this is right way to define inputs if I would like to make
#pixel based classification model
with tf.name_scope("inputs"):
    X = tf.placeholder(np.float32, shape=[None, n_channels], name="X")
    X_reshaped = tf.reshape(X, shape=[-1, height, width, n_channels])
    y = tf.placeholder(tf.int32, shape=[None], name="y")

dnn_outputs = dnn(X)

logits = tf.layers.dense(dnn_outputs, n_outputs, kernel_initializer=he_init, name="logits")
Y_proba = tf.nn.softmax(logits, name="Y_proba")

In [ ]:
learning_rate = 0.01

xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy, name="loss")

optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(loss, name="training_op")

correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, np.float32), name="accuracy")

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [ ]:
validlen = int(len(X_train) * 0.1)

X_valid = X_train[:validlen]
X_train = X_train[validlen:]
y_valid = y_train[:validlen]
y_train = y_train[validlen:]
#valied length is used primarily for implementing early stopping

In [ ]:
y_train[2]

0

In [ ]:
# n_epochs = 500
# batch_size = 5000

# max_checks_without_progress = 200
# checks_without_progress = 0
# best_loss = np.infty

# with tf.Session() as sess:
#     init.run()

#     for epoch in range(n_epochs):
#         rnd_idx = np.random.permutation(len(X_train))
#         for rnd_indices in np.array_split(rnd_idx, len(X_train) // batch_size):
# #             X_batch, y_batch = X_train[rnd_indices], y_train[rnd_indices]
#             X_batch = []
#             y_batch = []
#             for i in rnd_indices:
#                 X_batch.append(X_train[i])
#                 y_batch.append(y_train[i])
#             sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
#         loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_valid, y: y_valid})
#         if loss_val < best_loss:
#             save_path = saver.save(sess, "./simple_dnn_vege.ckpt")
#             best_loss = loss_val
#             checks_without_progress = 0
#         else:
#             checks_without_progress += 1
#             if checks_without_progress > max_checks_without_progress:
#                 print("Early stopping!")
#                 break
#         print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
#             epoch, loss_val, best_loss, acc_val * 100))

# with tf.Session() as sess:
#     saver.restore(sess, "./simple_dnn_vege.ckpt")
#     acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
#     print("Final test accuracy: {:.2f}%".format(acc_test * 100))

1. error from previous cell:
TypeError: only integer scalar arrays can be converted to a scalar index
Solution:
            X_batch = []
            y_batch = []
            for i in rnd_indices:
                X_batch.append(X_train[i])
                y_batch.append(y_train[i])

- TEST ON vege_1_1 does your code in execution phase does the same as theirs!

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.exceptions import NotFittedError

class DNNClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, n_hidden_layers=5, n_neurons=100, optimizer_class=tf.train.AdamOptimizer,
                 learning_rate=0.01, batch_size=20, activation=tf.nn.elu, initializer=he_init,
                 batch_norm_momentum=None, dropout_rate=None, random_state=None):
        """Initialize the DNNClassifier by simply storing all the hyperparameters."""
        self.n_hidden_layers = n_hidden_layers
        self.n_neurons = n_neurons
        self.optimizer_class = optimizer_class
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.activation = activation
        self.initializer = initializer
        self.batch_norm_momentum = batch_norm_momentum
        self.dropout_rate = dropout_rate
        self.random_state = random_state
        self._session = None

    def _dnn(self, inputs):
        """Build the hidden layers, with support for batch normalization and dropout."""
        for layer in range(self.n_hidden_layers):
            if self.dropout_rate:
                inputs = tf.layers.dropout(inputs, self.dropout_rate, training=self._training)
            inputs = tf.layers.dense(inputs, self.n_neurons,
                                     kernel_initializer=self.initializer,
                                     name="hidden%d" % (layer + 1))
            if self.batch_norm_momentum:
                inputs = tf.layers.batch_normalization(inputs, momentum=self.batch_norm_momentum,
                                                       training=self._training)
            inputs = self.activation(inputs, name="hidden%d_out" % (layer + 1))
        return inputs

    def _build_graph(self, n_inputs, n_outputs):
        """Build the same model as earlier"""
        if self.random_state is not None:
            tf.set_random_seed(self.random_state)
            np.random.seed(self.random_state)

        X = tf.placeholder(np.float32, shape=(None, n_channels), name="X")
        #here should go None, n_inputs for shape, but I have hardcoded it temporarily
        y = tf.placeholder(tf.int32, shape=(None), name="y")

        if self.batch_norm_momentum or self.dropout_rate:
            self._training = tf.placeholder_with_default(False, shape=(), name='training')
        else:
            self._training = None

        dnn_outputs = self._dnn(X)

        logits = tf.layers.dense(dnn_outputs, n_outputs, kernel_initializer=he_init, name="logits")
        Y_proba = tf.nn.softmax(logits, name="Y_proba")

        xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                                  logits=logits)
        loss = tf.reduce_mean(xentropy, name="loss")

        optimizer = self.optimizer_class(learning_rate=self.learning_rate)
        training_op = optimizer.minimize(loss)

        correct = tf.nn.in_top_k(logits, y, 1)
        accuracy = tf.reduce_mean(tf.cast(correct, np.float32), name="accuracy")

        init = tf.global_variables_initializer()
        saver = tf.train.Saver()

        # Make the important operations available easily through instance variables
        self._X, self._y = X, y
        self._Y_proba, self._loss = Y_proba, loss
        self._training_op, self._accuracy = training_op, accuracy
        self._init, self._saver = init, saver

    def close_session(self):
        if self._session:
            self._session.close()

    def _get_model_params(self):
        """Get all variable values (used for early stopping, faster than saving to disk)"""
        with self._graph.as_default():
            gvars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
        return {gvar.op.name: value for gvar, value in zip(gvars, self._session.run(gvars))}

    def _restore_model_params(self, model_params):
        """Set all variables to the given values (for early stopping, faster than loading from disk)"""
        gvar_names = list(model_params.keys())
        assign_ops = {gvar_name: self._graph.get_operation_by_name(gvar_name + "/Assign")
                      for gvar_name in gvar_names}
        init_values = {gvar_name: assign_op.inputs[1] for gvar_name, assign_op in assign_ops.items()}
        feed_dict = {init_values[gvar_name]: model_params[gvar_name] for gvar_name in gvar_names}
        self._session.run(assign_ops, feed_dict=feed_dict)
    def fit(self, X, y, n_epochs=200, X_valid=None, y_valid=None):
        """Fit the model to the training set. If X_valid and y_valid are provided, use early stopping."""
        self.close_session()

        # infer n_inputs and n_outputs from the training set.
        n_inputs = X.shape[1]
        self.classes_ = np.unique(y)
        n_outputs = len(self.classes_)
        
        # Translate the labels vector to a vector of sorted class indices, containing
        # integers from 0 to n_outputs - 1.
        # For example, if y is equal to [8, 8, 9, 5, 7, 6, 6, 6], then the sorted class
        # labels (self.classes_) will be equal to [5, 6, 7, 8, 9], and the labels vector
        # will be translated to [3, 3, 4, 0, 2, 1, 1, 1]
        self.class_to_index_ = {label: index
                                for index, label in enumerate(self.classes_)}
        y = np.array([self.class_to_index_[label]
                      for label in y], dtype=np.int32)
        
        self._graph = tf.Graph()
        with self._graph.as_default():
            self._build_graph(n_inputs, n_outputs)
            # extra ops for batch normalization
            extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

        # needed in case of early stopping
        max_checks_without_progress = 50
        checks_without_progress = 0
        best_loss = np.infty
        best_params = None
        
        # Now train the model!
        self._session = tf.Session(graph=self._graph)
        with self._session.as_default() as sess:
            self._init.run()
            for epoch in range(n_epochs):
                rnd_idx = np.random.permutation(len(X))
#                 import pdb
                for rnd_indices in np.array_split(rnd_idx, len(X) // self.batch_size):
#                     pdb.set_trace()
#                     X_batch, y_batch = X_train[rnd_indices], y_train[rnd_indices]
                    X_batch = []
                    y_batch = []
                    for i in rnd_indices:
                        X_batch.append(X_train[i])
                        y_batch.append(y_train[i])
                    X_batch = np.array(X_batch).astype(np.float32)
#                     X_batch = tf.convert_to_tensor(X_batch, np.float32)
                    y_batch = np.array(y_batch).astype(np.float32)
#                     y_batch = tf.convert_to_tensor(y_batch, np.float32)
#                     pdb.set_trace()
#                     sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
                    feed_dict = {self._X: X_batch, self._y: y_batch}
                    if self._training is not None:
                        feed_dict[self._training] = True
                    sess.run(self._training_op, feed_dict=feed_dict)
                    if extra_update_ops:
                        sess.run(extra_update_ops, feed_dict=feed_dict)
                if X_valid is not None and y_valid is not None:
                    loss_val, acc_val = sess.run([self._loss, self._accuracy],
                                                 feed_dict={self._X: X_valid,
                                                            self._y: y_valid})
                    if loss_val < best_loss:
                        best_params = self._get_model_params()
                        best_loss = loss_val
                        checks_without_progress = 0
                    else:
                        checks_without_progress += 1
                    print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
                        epoch, loss_val, best_loss, acc_val * 100))
                    if checks_without_progress > max_checks_without_progress:
                        print("Early stopping!")
                        break
                else:
                    loss_train, acc_train = sess.run([self._loss, self._accuracy],
                                                     feed_dict={self._X: X_batch,
                                                                self._y: y_batch})
                    print("{}\tLast training batch loss: {:.6f}\tAccuracy: {:.2f}%".format(
                        epoch, loss_train, acc_train * 100))
            # If we used early stopping then rollback to the best model found
            if best_params:
                self._restore_model_params(best_params)
            return self

    def predict_proba(self, X):
        if not self._session:
            raise NotFittedError("This %s instance is not fitted yet" % self.__class__.__name__)
        with self._session.as_default() as sess:
            return self._Y_proba.eval(feed_dict={self._X: X})

    def predict(self, X):
        class_indices = np.argmax(self.predict_proba(X), axis=1)
        return np.array([[self.classes_[class_index]]
                         for class_index in class_indices], np.int32)

    def save(self, path):
        self._saver.save(self._session, path)

Next cell is used for importing randomizedsearchcv

TRY ALSO USING GRID SEARCH!

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

def leaky_relu(alpha=0.01):
    def parametrized_leaky_relu(z, name=None):
        return tf.maximum(alpha * z, z, name=name)
    return parametrized_leaky_relu

param_distribs = {
    "n_neurons": [20, 50, 100, 150, 200],
    "batch_size": [500, 1000, 2000, 5000],
    "learning_rate": [0.02, 0.05, 0.1],
    "activation": [tf.nn.relu, tf.nn.elu, leaky_relu(alpha=0.01), leaky_relu(alpha=0.1)],
    "batch_norm_momentum": [0.9, 0.95, 0.98, 0.99],
    "dropout_rate": [0.2, 0.4, 0.5, 0.6],
    "n_hidden_layers": [2, 3, 4, 5, 8, 10],
}

In [ ]:
# %%pixie_debugger
rnd_search = RandomizedSearchCV(DNNClassifier(random_state=17), param_distribs, n_iter=50,
                                cv=3, random_state=17, verbose=2)
# pdb.set_trace()
rnd_search.fit(X_train, y_train, X_valid=X_valid, y_valid=y_valid, n_epochs=200)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] n_neurons=150, n_hidden_layers=2, learning_rate=0.1, dropout_rate=0.2, batch_size=2000, batch_norm_momentum=0.99, activation=<function elu at 0x7f93a9f21950> 
0	Validation loss: 0.707068	Best loss: 0.707068	Accuracy: 76.12%
1	Validation loss: 0.705499	Best loss: 0.705499	Accuracy: 76.12%
2	Validation loss: 0.705151	Best loss: 0.705151	Accuracy: 76.12%
3	Validation loss: 0.714578	Best loss: 0.705151	Accuracy: 76.12%
4	Validation loss: 0.707459	Best loss: 0.705151	Accuracy: 76.12%
5	Validation loss: 0.708932	Best loss: 0.705151	Accuracy: 76.12%
6	Validation loss: 0.707173	Best loss: 0.705151	Accuracy: 76.12%
7	Validation loss: 0.710495	Best loss: 0.705151	Accuracy: 76.12%
8	Validation loss: 0.730948	Best loss: 0.705151	Accuracy: 76.12%
9	Validation loss: 0.726233	Best loss: 0.705151	Accuracy: 76.12%
10	Validation loss: 0.705576	Best loss: 0.705151	Accuracy: 76.12%
11	Validation loss: 0.713820	Best loss: 0.705151	Accuracy: 76.12%
12	Validation loss: 0.707560	Best loss: 0.705151	Accur

122	Validation loss: 0.720272	Best loss: 0.704060	Accuracy: 76.12%
123	Validation loss: 1.229696	Best loss: 0.704060	Accuracy: 76.11%
Early stopping!
[CV]  n_neurons=150, n_hidden_layers=2, learning_rate=0.1, dropout_rate=0.2, batch_size=2000, batch_norm_momentum=0.99, activation=<function elu at 0x7f93a9f21950>, total=17.9min
[CV] n_neurons=150, n_hidden_layers=2, learning_rate=0.1, dropout_rate=0.2, batch_size=2000, batch_norm_momentum=0.99, activation=<function elu at 0x7f93a9f21950> 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 18.0min remaining:    0.0s


...

65	Validation loss: 0.713176	Best loss: 0.704233	Accuracy: 76.12%
66	Validation loss: 0.707524	Best loss: 0.704233	Accuracy: 76.12%
67	Validation loss: 0.705679	Best loss: 0.704233	Accuracy: 76.12%
68	Validation loss: 0.709862	Best loss: 0.704233	Accuracy: 76.12%
69	Validation loss: 0.713713	Best loss: 0.704233	Accuracy: 76.12%
Early stopping!
[CV]  n_neurons=150, n_hidden_layers=2, learning_rate=0.1, dropout_rate=0.2, batch_size=2000, batch_norm_momentum=0.99, activation=<function elu at 0x7f93a9f21950>, total=10.4min
[CV] n_neurons=50, n_hidden_layers=2, learning_rate=0.05, dropout_rate=0.2, batch_size=2000, batch_norm_momentum=0.95, activation=<function relu at 0x7f93a9f4ec80> 
0	Validation loss: 0.705316	Best loss: 0.705316	Accuracy: 76.12%
1	Validation loss: 0.704934	Best loss: 0.704934	Accuracy: 76.12%
2	Validation loss: 0.704399	Best loss: 0.704399	Accuracy: 76.12%
3	Validation loss: 0.705243	Best loss: 0.704399	Accuracy: 76.12%
4	Validation loss: 0.705281	Best loss: 0.704399	Ac

32	Validation loss: 0.703515	Best loss: 0.703189	Accuracy: 76.12%
33	Validation loss: 0.705410	Best loss: 0.703189	Accuracy: 76.12%
34	Validation loss: 0.704640	Best loss: 0.703189	Accuracy: 76.12%
35	Validation loss: 0.703867	Best loss: 0.703189	Accuracy: 76.12%
36	Validation loss: 0.705479	Best loss: 0.703189	Accuracy: 76.12%
37	Validation loss: 0.704491	Best loss: 0.703189	Accuracy: 76.12%
38	Validation loss: 0.703744	Best loss: 0.703189	Accuracy: 76.12%
39	Validation loss: 0.703061	Best loss: 0.703061	Accuracy: 76.12%
40	Validation loss: 0.703844	Best loss: 0.703061	Accuracy: 76.12%
41	Validation loss: 0.704648	Best loss: 0.703061	Accuracy: 76.12%
42	Validation loss: 0.704093	Best loss: 0.703061	Accuracy: 76.12%
43	Validation loss: 0.704125	Best loss: 0.703061	Accuracy: 76.12%
44	Validation loss: 0.704175	Best loss: 0.703061	Accuracy: 76.12%
45	Validation loss: 0.704745	Best loss: 0.703061	Accuracy: 76.12%
46	Validation loss: 0.703663	Best loss: 0.703061	Accuracy: 76.12%
47	Validat

60	Validation loss: 0.703509	Best loss: 0.703135	Accuracy: 76.12%
61	Validation loss: 0.706996	Best loss: 0.703135	Accuracy: 76.12%
62	Validation loss: 0.703331	Best loss: 0.703135	Accuracy: 76.12%
63	Validation loss: 0.703376	Best loss: 0.703135	Accuracy: 76.12%
64	Validation loss: 0.705414	Best loss: 0.703135	Accuracy: 76.12%
65	Validation loss: 0.703862	Best loss: 0.703135	Accuracy: 76.12%
66	Validation loss: 0.704443	Best loss: 0.703135	Accuracy: 76.12%
67	Validation loss: 0.704066	Best loss: 0.703135	Accuracy: 76.12%
68	Validation loss: 0.706330	Best loss: 0.703135	Accuracy: 76.12%
69	Validation loss: 0.704251	Best loss: 0.703135	Accuracy: 76.12%
70	Validation loss: 0.704971	Best loss: 0.703135	Accuracy: 76.12%
71	Validation loss: 0.704296	Best loss: 0.703135	Accuracy: 76.12%
72	Validation loss: 0.703212	Best loss: 0.703135	Accuracy: 76.12%
73	Validation loss: 0.704980	Best loss: 0.703135	Accuracy: 76.12%
74	Validation loss: 0.704249	Best loss: 0.703135	Accuracy: 76.12%
75	Validat

88	Validation loss: 0.703053	Best loss: 0.701595	Accuracy: 76.14%
89	Validation loss: 0.703347	Best loss: 0.701595	Accuracy: 76.14%
90	Validation loss: 0.702817	Best loss: 0.701595	Accuracy: 76.14%
91	Validation loss: 0.702325	Best loss: 0.701595	Accuracy: 76.13%
92	Validation loss: 0.702247	Best loss: 0.701595	Accuracy: 76.14%
93	Validation loss: 0.702783	Best loss: 0.701595	Accuracy: 76.14%
Early stopping!
[CV]  n_neurons=100, n_hidden_layers=8, learning_rate=0.02, dropout_rate=0.2, batch_size=2000, batch_norm_momentum=0.95, activation=<function leaky_relu.<locals>.parametrized_leaky_relu at 0x7f921a9c57b8>, total=31.2min
[CV] n_neurons=100, n_hidden_layers=8, learning_rate=0.02, dropout_rate=0.2, batch_size=2000, batch_norm_momentum=0.95, activation=<function leaky_relu.<locals>.parametrized_leaky_relu at 0x7f921a9c57b8> 
0	Validation loss: 0.705473	Best loss: 0.705473	Accuracy: 76.12%
1	Validation loss: 0.703205	Best loss: 0.703205	Accuracy: 76.12%
2	Validation loss: 0.703272	Best 

112	Validation loss: 0.704132	Best loss: 0.702140	Accuracy: 76.14%
113	Validation loss: 0.703458	Best loss: 0.702140	Accuracy: 76.13%
114	Validation loss: 0.703370	Best loss: 0.702140	Accuracy: 76.14%
115	Validation loss: 0.702750	Best loss: 0.702140	Accuracy: 76.14%
116	Validation loss: 0.702610	Best loss: 0.702140	Accuracy: 76.13%
117	Validation loss: 0.703489	Best loss: 0.702140	Accuracy: 76.14%
118	Validation loss: 0.703601	Best loss: 0.702140	Accuracy: 76.14%
119	Validation loss: 0.703108	Best loss: 0.702140	Accuracy: 76.13%
120	Validation loss: 0.702340	Best loss: 0.702140	Accuracy: 76.13%
121	Validation loss: 0.703991	Best loss: 0.702140	Accuracy: 76.13%
122	Validation loss: 0.703505	Best loss: 0.702140	Accuracy: 76.14%
123	Validation loss: 0.702521	Best loss: 0.702140	Accuracy: 76.13%
124	Validation loss: 0.702994	Best loss: 0.702140	Accuracy: 76.14%
125	Validation loss: 0.703797	Best loss: 0.702140	Accuracy: 76.13%
126	Validation loss: 0.703726	Best loss: 0.702140	Accuracy: 76

Errors in previous cell:
1. error
row 3: unhashable type: 'numpy.ndarray'
possible solutions:
https://datascience.stackexchange.com/questions/39831/typeerror-unhashable-type-numpy-ndarray?rq=1
"The problem is that you're passing a list of numpy arrays to the mode function. It requires either a single list of values, or a single numpy array with values (basically any single container will do, but seemingly not a list of arrays)." 

-- try to find where list of arrays is passed but it shouldn't be - considering this whole error message maybe there is problem with passing X_batch and y_batch values to the feed_dict